In [148]:
from genetic_algorithm import *
import re
import json
import numpy as np
from numpy.random import default_rng
from collections import Counter
import copy
import matplotlib.pyplot as plt
from itertools import groupby

## hexadecimal generator

In [104]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [105]:
## world size
world_size_x = 128
world_size_y = 128

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)


In [106]:
individuals_sum_dup = sum_duplicated_neurons(individuals)

In [107]:
individuals_sum_dup

{0: {'mid1mid0': ['mid1', 'mid0', 1.13486952540821],
  'mid1out0': ['mid1', 'out0', 2.3404242331756446],
  'in2mid2': ['in2', 'mid2', 1.2403479322447735],
  'mid2mid0': ['mid2', 'mid0', -2.447978025331909],
  'mid2mid2': ['mid2', 'mid2', 1.6388219136273463],
  'mid2mid1': ['mid2', 'mid1', 1.250846940332672],
  'in0mid0': ['in0', 'mid0', 3.156538989775675],
  'mid1out1': ['mid1', 'out1', -3.569296505417366],
  'in0out1': ['in0', 'out1', -3.273004730657714],
  'mid2out0': ['mid2', 'out0', -3.7868457195177783],
  'mid0mid1': ['mid0', 'mid1', 3.0093087135663055],
  'mid0out2': ['mid0', 'out2', -2.1627956661071264],
  'in1mid1': ['in1', 'mid1', 2.3538531970090033],
  'in0out2': ['in0', 'out2', 3.043125286128491]},
 1: {'mid2out1': ['mid2', 'out1', -3.156172745307493],
  'in2mid1': ['in2', 'mid1', 1.2403479322447735],
  'mid2out0': ['mid2', 'out0', -1.5421333740271632],
  'mid1mid1': ['mid1', 'mid1', -2.569449107279109],
  'mid0out1': ['mid0', 'out1', -2.9427743018464825],
  'in0mid1': ['in0

## initial brain and position generator

In [5]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## brain generator
for individual in individuals_sum_dup:
    
    ## init brain neuron generator
    print(individual)
    ## preprocessing
    edges = individuals_sum_dup[individual]
    edges = [tuple(edges[i]) for i in edges]
    edges_no_weight = [i[:-1] for i in edges]
    
    G = nx.MultiDiGraph()
    G.add_edges_from(edges)
    
    out_list = list(set([i[1] for i in edges if 'out' in i[1]]))
    init_list = list(set([i[0] for i in edges if 'in' in i[0]]))

    # generate dict of paths
    dic_of_paths = generate_dict_of_paths(out_list, init_list, G)

    lis = filtered_neurons_paths(dic_of_paths)
    
    dic = append_weight_to_neurons_in_path(lis, edges_no_weight, edges)
#     print(dic)
    
        
        

#     print(dic)
    print()

0
dic_list_plus_weight {'mid0': {'mid1': -2.308194719975584, 'mid0': 0.95309018770029, 'in0': -3.715672211200977}, 'mid1': 0.9895215628286798, 'out1': {'mid0': 1.3575461620631772}}
dic_list_plus_weight {'mid0': {'mid1': -2.308194719975584, 'mid0': 0.95309018770029, 'in0': -3.715672211200977}, 'mid1': 0.9895215628286798, 'out0': {'in1': -3.0837784220967497, 'mid0': -2.6151075843125287, 'in2': -2.4857012055547076}}
dic_list_plus_weight {'mid0': {'mid1': -2.308194719975584, 'mid0': 0.95309018770029, 'in0': -3.715672211200977}, 'mid1': 0.9895215628286798, 'out2': {'mid0': 1.3311765603540364}}

1
dic_list_plus_weight {'mid0': {'mid1': 0.6427590416603083, 'mid0': 1.0666259728368686, 'mid2': 1.8130321989928277}, 'mid1': {'in0': 2.1298336639707003, 'mid0': 1.3155501297115826}, 'mid2': {'mid1': -1.8760262475202198, 'in2': 2.867816267358462, 'mid2': -2.1082252403479322}, 'out1': {'mid0': 1.365969784831375, 'mid1': 0.9870288417518693}}


TypeError: unsupported operand type(s) for +=: 'float' and 'dict'

In [165]:
def remove_self_loop(dic):
    '''remove neurons if self looped ie: Amid->Bmid, Bmid->Amid'''
    for nr in dic:
        list_of_dup = []
        for key_1 in dic[nr]:
            for key_2 in dic[nr]:
                if key_1 != key_2 and sorted(key_1) == sorted(key_2):
                    list_of_dup.append(sorted([key_1, key_2]))
                    
        list_of_dup.sort()
        list_of_dup
        
    return dic

In [166]:
d = {'0':{'12':('1','2'),'23': ('2','3'),'34': ('3','4'),'32': ('3','2'), '43': ('4','3')}}
a, b= remove_self_loop(d)

In [170]:
b.sort()
b = list(b for b,_ in itertools.groupby(b))
for i in b:
    rand_int = random.randint(0, 1)
    del d['0'][i[rand_int]]

In [171]:
d

{'0': {'12': ('1', '2'), '23': ('2', '3'), '34': ('3', '4')}}

In [ ]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
for individual in individuals_sum_dup_no_self_loop:
    
    ## init brain neuron generator
    
    ## preprocessing
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    remove_mid_with_no_predecessor(edges) 
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
    mid_to_weight(dic)
    out_to_weight(dic)
       
    dic = {k:v for k, v in dic.items() if 'out' in k}    
    
    initial_dic_of_output_neurons[individual] = {}
    initial_dic_of_output_neurons[individual]['output'] = dic
    
    ## position
#     initial_dic_of_output_neurons[individual]['position'] = {}
#     initial_dic_of_output_neurons[individual]['position']['x'] = x[individual]
#     initial_dic_of_output_neurons[individual]['position']['y'] = y[individual] 
    
    ## position history
    initial_dic_of_output_neurons[individual]['history_position'] = [(x[individual], y[individual])]

In [ ]:
initial_dic_of_output_neurons

In [ ]:
a = [(3, 0),(4,1),(5,2)]

In [ ]:
def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

print(slope_intercept(3,0,2,1))

In [ ]:
a

In [ ]:
if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
print(x)

In [ ]:
a[-2][0]